In [30]:
#encoding=utf-8
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

In [31]:
# read files
path = '../../data/WillingToPay/'
data = pd.read_csv(path + 'cleaned_mock_up_data.csv', encoding='utf8')
print(data.shape)
data.head(10)

(215, 2)


,label,split_text
0,0,我 等 会 就 处理
1,1,我 现在 没 钱
2,2,这 太 多 了 能 减免 一点 吗
3,3,我 听不清
4,0,行 会 尽快 的
5,0,好 的 没 问题
6,0,明白 了 那 要 在 什么 时候 之前 还 呢
7,0,嗯 嗯 没 问题
8,0,我 可以 还 要 在 多久 之前 呢
9,0,我 过 几天 就 还


In [5]:
data.split_text.iloc[0]

'我   等 会   就   处理'

In [6]:
phrase_vectorizer = TfidfVectorizer(ngram_range=(1,3),
                                    strip_accents='unicode', 
                                    max_features=100000, 
                                    analyzer='word',
                                    sublinear_tf=True,
                                    token_pattern=r'\w{1,}')

print('fitting phrase')
phrase_vectorizer.fit(data.split_text)

print('transform phrase')
phrase = phrase_vectorizer.transform(data.split_text)

phrase

fitting phrase
transform phrase


<215x840 sparse matrix of type '<class 'numpy.float64'>'
	with 2173 stored elements in Compressed Sparse Row format>

# SVC

In [7]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC

l_svc = LinearSVC()
clf = CalibratedClassifierCV(l_svc) 
clf.fit(phrase, data.label)
print(clf.score(phrase, data.label))

1.0


In [8]:
print(clf.predict(phrase))

[0 1 2 3 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 3
 3 3 3 3 3 3 0 1 2 3 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2
 2 2 2 2 2 3 3 3 3 3 3 3 0 1 2 3 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 2 2 2 2 2 2 2 3 3 3 3 3 3 3 0 1 2 3 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 3 3 3 3 3 3 3 0 1 2 3 0 0 0 0 0 0 0 0 0
 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 3 3 3 3 3 3 3]


# logistic

In [9]:
from sklearn.linear_model import LogisticRegression

In [10]:
log_r = LogisticRegression()
log_r.fit(phrase, data.label)
print(log_r.score(phrase, data.label))

1.0


In [11]:
print(log_r.predict(phrase))

[0 1 2 3 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 3
 3 3 3 3 3 3 0 1 2 3 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2
 2 2 2 2 2 3 3 3 3 3 3 3 0 1 2 3 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 2 2 2 2 2 2 2 3 3 3 3 3 3 3 0 1 2 3 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 3 3 3 3 3 3 3 0 1 2 3 0 0 0 0 0 0 0 0 0
 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 3 3 3 3 3 3 3]


# LightGBM

In [12]:
import lightgbm as lgb

In [13]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(data.label.values)
onelabels = le.transform(data.label.values)

In [14]:
onelabels

array([0, 1, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 0, 1, 2,
       3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 0, 1, 2, 3, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 0, 1, 2, 3, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2,
       2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 0, 1, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
       2, 3, 3, 3, 3, 3, 3, 3])

In [15]:
from sklearn.preprocessing import MultiLabelBinarizer
multicoder = MultiLabelBinarizer()
lables = multicoder.fit_transform([data.label.values])

In [16]:
lables

array([[1, 1, 1, 1]])

In [17]:
data.label.values

array([0, 1, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 0, 1, 2,
       3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 0, 1, 2, 3, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 0, 1, 2, 3, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2,
       2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 0, 1, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
       2, 3, 3, 3, 3, 3, 3, 3])

In [18]:
params = {
    'learning_rate': 0.2,
    'num_iterations':1000,
    'application': 'multiclassova',
    'num_class': 4,
    'num_leaves': 31,
    'verbosity': -1,
    'metric': 'multi_error',
    'data_random_seed': 2,
#     'bagging_fraction': 0.8,
#     'feature_fraction': 0.6,
    'nthread': 4,
    'lambda_l1': 1,
    'lambda_l2': 1,
    'early_stopping_round':200
} 

# lgbm_train = lgb.Dataset(phrase, data.label)
lgbm_train = lgb.Dataset(phrase, onelabels)
lgbm_val = lgb.Dataset(phrase, onelabels)
lgbm_model = lgb.train(params,lgbm_train,valid_sets=lgbm_val, verbose_eval=5)

Training until validation scores don't improve for 200 rounds.
[5]	valid_0's multi_error: 0.316279
[10]	valid_0's multi_error: 0.283721
[15]	valid_0's multi_error: 0.269767
[20]	valid_0's multi_error: 0.255814
[25]	valid_0's multi_error: 0.2
[30]	valid_0's multi_error: 0.195349
[35]	valid_0's multi_error: 0.195349
[40]	valid_0's multi_error: 0.190698
[45]	valid_0's multi_error: 0.190698
[50]	valid_0's multi_error: 0.190698
[55]	valid_0's multi_error: 0.186047
[60]	valid_0's multi_error: 0.186047
[65]	valid_0's multi_error: 0.181395
[70]	valid_0's multi_error: 0.172093
[75]	valid_0's multi_error: 0.172093
[80]	valid_0's multi_error: 0.162791
[85]	valid_0's multi_error: 0.162791
[90]	valid_0's multi_error: 0.162791
[95]	valid_0's multi_error: 0.162791
[100]	valid_0's multi_error: 0.162791
[105]	valid_0's multi_error: 0.153488
[110]	valid_0's multi_error: 0.153488
[115]	valid_0's multi_error: 0.153488
[120]	valid_0's multi_error: 0.148837
[125]	valid_0's multi_error: 0.144186
[130]	valid_

/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [19]:
print(lgbm_model.predict(phrase))

[[ 0.56565755  0.28009692  0.01069672  0.01562528]
 [ 0.03800355  0.84410063  0.00340963  0.25112153]
 [ 0.03635962  0.0268971   0.97890555  0.01673974]
 [ 0.18354215  0.29815331  0.007096    0.36961908]
 [ 0.35698152  0.17418407  0.33471278  0.01539359]
 [ 0.69630278  0.17418407  0.08052549  0.01539359]
 [ 0.62616213  0.04966928  0.18622265  0.08481559]
 [ 0.69630278  0.04575722  0.08052549  0.11876347]
 [ 0.76174136  0.27360664  0.01934429  0.00922768]
 [ 0.49161804  0.44897036  0.0064571   0.02519001]
 [ 0.58595394  0.25268717  0.00517586  0.06379722]
 [ 0.56565755  0.44460304  0.0078099   0.0096648 ]
 [ 0.33465548  0.72479484  0.0064571   0.02071937]
 [ 0.79175331  0.01889279  0.0061422   0.13389175]
 [ 0.57441868  0.32228656  0.00517586  0.06379722]
 [ 0.02237613  0.99024345  0.00690448  0.00494306]
 [ 0.01960483  0.91607432  0.01915681  0.01438757]
 [ 0.18354215  0.83984511  0.007096    0.03915252]
 [ 0.05445817  0.92901911  0.00340963  0.07831019]
 [ 0.07304749  0.97641828  0.02

In [20]:
# lgbm_model.predict(phrase)

# Save

In [21]:
import pickle

# save tfidf
pickle.dump(phrase_vectorizer, open("../../savedModel/WillingToPay/tfidf.pickle", "wb"))
# pickle.dump(train_comment_features, open("train_comment_features.pickle", "wb"))
# pickle.dump(test_comment_features, open("test_comment_features.pickle", "wb"))

# save linear svc
pickle.dump(clf, open("../../savedModel/WillingToPay/LinearSVC.pickle", "wb"))
# save logistic
pickle.dump(log_r, open("../../savedModel/WillingToPay/Logistic.pickle", "wb"))
# save lightGBM
pickle.dump(lgbm_model, open("../../savedModel/WillingToPay/Lgbm.pickle", "wb"))

# Test

In [22]:
test = phrase_vectorizer.transform(['真的 没有 钱'])

In [23]:
clf.predict_proba(test) # linear svc

array([[ 0.00595701,  0.97996783,  0.00984511,  0.00423006]])

In [24]:
log_r.predict_proba(test) # logistic

array([[ 0.08694929,  0.76574203,  0.07715617,  0.07015252]])

In [25]:
lgbm_model.predict(test) # light gbm

array([[ 0.00518806,  0.98725149,  0.15624742,  0.02114158]])

# labeling

In [26]:
# basic logic: find the max probability of 3 models, if it is larger than threshold, return the corresponding label, otherwise, return 2 (others).
result = np.vstack((clf.predict_proba(test),log_r.predict_proba(test),lgbm_model.predict(test)))
pos = np.where(result == np.max(result))

threshold = 0.7
if np.max(result)<threshold:
    label = 2
else:
    label = pos[1]
    label = label[0]
    
print('label=',label)
print('prob=',np.max(result))

label= 1
prob= 0.987251485545


# save the whole thing

In [27]:
# import jieba
# import numpy as np

# class WillingToPay:
    
#     def __init__(self, **model):
#         """
#         suggested parameters:
#         svc, logistic, lightgbm, jieba_path,tfidf
#         """
#         self._load_model(**model)
        
#     def _load_model(self,**model):
#         self.svc = model.get('svc')
#         self.logistic = model.get('logistic')
#         self.lightgbm = model.get('lightgbm')
#         self.tfidf = model.get('tfidf')
#         # load jieba
#         jieba_path = model.get('jieba_path')
#         if jieba_path is not None:
#             jieba.load_userdict(jieba_path)
        
        
#     def classify(self, sentence):
#         sentence = jieba.cut(sentence, cut_all = False)
#         sentence = ' '.join(sentence)
#         matrix = self.tfidf.transform([sentence])
#         result = np.vstack((self.svc.predict_proba(matrix),
#                             self.logistic.predict_proba(matrix),
#                             self.lightgbm.predict(matrix)))
#         max_pred = np.max(result, axis=0)
#         max_arg = np.argmax(max_pred)
#         threshold = 0.6
#         if np.max(max_pred<threshold):
#             label = 3
#         else:
#             label = max_arg
#         return (label, np.max(max_pred))
     
        

In [28]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
from WillingToPay_py import WillingToPay
ifk = WillingToPay(svc=clf, logistic=log_r, lightgbm=lgbm_model, tfidf=phrase_vectorizer, jieba_path='../WordCut/userdict.txt')

pickle.dump(ifk, open("../../savedModel/WillingToPay/WillingToPay.pickle", "wb"))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.465 seconds.
Prefix dict has been built succesfully.
